# Boston Crimes


In this project we are going to create a database for storing data about crimes in boston. We are going to optimize data types and create a schema for specific users to access the database. This will help us understand data base creations in PostgreSQL and authorization hierarchy.

Let's first create our database. After it is created, we will create a schema for it. We can leave the connection closed for the rest of the project

In [1]:
import psycopg2 
import csv

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Create new database
conn = psycopg2.connect(dbname = 'postgres', user = 'postgres', password = '')
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db")
conn.autocommit = False
conn.close()

DuplicateDatabase: database "crime_db" already exists


In [5]:
conn = psycopg2.connect(dbname ='crime_db', user = 'postgres', password = '')
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")

Let's make sure our schema has been created.

In [6]:
cur.execute("select schema_name from information_schema.schemata;")
schema = cur.fetchall()
print(schema)

[('pg_toast',), ('pg_temp_1',), ('pg_toast_temp_1',), ('pg_catalog',), ('information_schema',), ('public',), ('crimes',)]


We can now see it's open. Now, let's read the file and save the headers and rows.

In [7]:
filename = 'boston.csv'
with open(filename) as file:
    reader = csv.reader(file)
    rows = list(reader)
    header = rows[0]
    rows = rows[1:]

In [8]:
print(header)
print('\n')
print(rows[0])

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


['1', '619', 'LARCENY ALL OTHERS', '9/2/2018', 'Sunday', '42.35779134', '-71.13937053']


### Analyzing data
Now that we can properly read our file, let's start analyzing our databy looking at the amount of unique values per column.

In [9]:
def get_col_values(file, index):
    col_set = set()
    for row in rows:
        col_set.add(row[index])        
    return len(col_set)


In [10]:
for column in range(len(header)):
    value = get_col_values('boston.csv', column)
    print("The number of unique values in column #" + str(column) + " " 
          + "with column name " + str(header[column]) + " " + "is: " + str(value))

The number of unique values in column #0 with column name incident_number is: 298329
The number of unique values in column #1 with column name offense_code is: 219
The number of unique values in column #2 with column name description is: 239
The number of unique values in column #3 with column name date is: 1177
The number of unique values in column #4 with column name day_of_the_week is: 7
The number of unique values in column #5 with column name lat is: 18177
The number of unique values in column #6 with column name long is: 18177


Now we can identify which columns have the most unique values. We can see that day_of_week only has 7 unique values (which are the days of the week), and also offense_code and description do not have that many unique values. 

Let's now look at the maximum number of characters per column.
Our columns are
- incident_number
- offense_code
- description
- date
- day_of_the_week
- lat
- long

From this columns, the only one that might have a large length should be `description`. Let's see how big is the largest value.


In [11]:
max_length = len(rows[0][2])
for row in rows:
    val = len(row[2])
    if max_length < val:
        max_length = val
print(max_length)

58


Now we know that the `description` size should be 58, we'll round it to 80. Let's start creating our database.

### Creation of database
This will be the steps to create our database

- Create the table
    - We wil also create an enumerator for the days of the week.
- Insert the data into the table
- Create two different type of groups
    - readonly: Commands available are
        - SELECT
    - readwrite: Commands available are
        - SELECT
        - INSERT
        - DELETE
        - UPDATE
- Create some test users and assign them to both groups        

In [12]:
cur.execute("CREATE TYPE WEEK AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")
cur.execute("""
            CREATE TABLE crimes.boston_crimes (
            incident_number INTEGER PRIMARY KEY,
            offense_code INTEGER, 
            description VARCHAR(80),
            date DATE,
            day_of_the_week WEEK,
            lat DECIMAL,
            long DECIMAL
            );
            """
           )
with open("boston.csv") as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)


In [13]:
cur.execute("REVOKE ALL ON SCHEMA PUBLIC FROM PUBLIC;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM PUBLIC;")

In [14]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")



In [15]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("GRANT readwrite TO data_scientist;")

In [17]:
conn.commit()
conn.close()

### Testing
Now that our database is ready, we are ready to do some testing. We are going to test to see that the privileges are set correctly.

In [20]:
conn = psycopg2.connect(dbname = 'crime_db', user = 'postgres', password = 'david123.D')
cur = conn.cursor()



In [21]:
cur.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee = 'readwrite';")
readwrite = cur.fetchall()
print(readwrite)

[('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]


In [22]:
cur.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee = 'readonly';")
readonly = cur.fetchall()
print(readonly)

[('readonly', 'SELECT')]


Looks like the privileges are correct. We are done we the project :) .

In [23]:
conn.close()